# Example 8: Genetic Algorithm Search

Examples 1--5 use the exhaustive **Generate-and-Test** workflow: enumerate every k-combination, score each, pick the best. This works well for small combination spaces (e.g., 12-choose-4 = 495 combos), but becomes infeasible for large problems (e.g., 52-choose-8 = 752M combos).

This notebook introduces two scalable alternatives that stay within the Generate-and-Test paradigm:

| Algorithm | Idea | Use case |
|-----------|------|----------|
| **Random Sampling** | Evaluate N random valid combinations | Cheap baseline, useful for benchmarking |
| **Genetic Algorithm** | Evolve a population of candidates over generations | Scalable search, finds near-optimal solutions |

Both algorithms reuse the same `ObjectiveSet` for scoring and `SelectionPolicy` for final winner selection. The GA additionally uses a pluggable `FitnessStrategy` for tournament selection during evolution.

In [ ]:
import time

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio; pio.renderers.default = 'notebook_connected'

import energy_repset as rep
import energy_repset.diagnostics as diag

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
df_raw = pd.read_csv(url, index_col=0, parse_dates=True).rename_axis('variable', axis=1)
df_raw = df_raw.drop('prices', axis=1)

context = rep.ProblemContext(df_raw=df_raw, slicer=rep.TimeSlicer(unit="month"))

objective_set = rep.ObjectiveSet({
    'wasserstein': (1.0, rep.WassersteinFidelity()),
    'correlation': (0.5, rep.CorrelationFidelity()),
})

k = 4
print(f"Selecting {k} representative months from {len(context.get_unique_slices())} candidates")

---
## 1. Exhaustive Baseline

With 12 months and k=4, the exhaustive search evaluates all C(12, 4) = 495 combinations. This is our ground-truth reference.

In [ ]:
workflow_exhaustive = rep.Workflow(
    feature_engineer=rep.StandardStatsFeatureEngineer(),
    search_algorithm=rep.ObjectiveDrivenCombinatorialSearchAlgorithm(
        objective_set=objective_set,
        selection_policy=rep.WeightedSumPolicy(),
        combination_generator=rep.ExhaustiveCombiGen(k=k),
    ),
    representation_model=rep.UniformRepresentationModel(),
)

t0 = time.perf_counter()
result_exhaustive = rep.RepSetExperiment(context, workflow_exhaustive).run()
time_exhaustive = time.perf_counter() - t0

print(f"Selection: {result_exhaustive.selection}")
print(f"Scores:    {result_exhaustive.scores}")
print(f"Time:      {time_exhaustive:.2f}s")

---
## 2. Random Sampling

`RandomSamplingSearch` draws N random valid k-combinations, evaluates each, and picks the best. It's a one-line swap for the search algorithm — everything else stays the same.

In [ ]:
workflow_random = rep.Workflow(
    feature_engineer=rep.StandardStatsFeatureEngineer(),
    search_algorithm=rep.RandomSamplingSearch(
        objective_set=objective_set,
        selection_policy=rep.WeightedSumPolicy(),
        combination_generator=rep.ExhaustiveCombiGen(k=k),
        n_samples=200,
        seed=42,
    ),
    representation_model=rep.UniformRepresentationModel(),
)

t0 = time.perf_counter()
result_random = rep.RepSetExperiment(context, workflow_random).run()
time_random = time.perf_counter() - t0

print(f"Selection: {result_random.selection}")
print(f"Scores:    {result_random.scores}")
print(f"Time:      {time_random:.2f}s")

In [ ]:
fig = diag.ResponsibilityBars().plot(result_random.weights, show_uniform_reference=True)
fig.update_layout(title='Random Sampling: Responsibility Weights')
fig.show()

---
## 3. Genetic Algorithm (Weighted Sum Fitness)

The `GeneticAlgorithmSearch` evolves a population of candidate selections over multiple generations. Each generation:

1. **Evaluate** all individuals with the ObjectiveSet
2. **Rank** individuals using a `FitnessStrategy` (here: weighted sum)
3. **Select** parents via tournament selection
4. **Crossover** parent gene pools to create offspring
5. **Mutate** offspring by swapping random genes
6. **Preserve** elite individuals unchanged

The default `WeightedSumFitness` scalarizes multi-objective scores into a single fitness value using the weights from the ObjectiveSet.

In [ ]:
workflow_ga = rep.Workflow(
    feature_engineer=rep.StandardStatsFeatureEngineer(),
    search_algorithm=rep.GeneticAlgorithmSearch(
        objective_set=objective_set,
        selection_policy=rep.WeightedSumPolicy(),
        combination_generator=rep.ExhaustiveCombiGen(k=k),
        fitness_strategy=rep.WeightedSumFitness(),
        population_size=30,
        n_generations=50,
        mutation_rate=0.15,
        crossover_rate=0.8,
        elite_fraction=0.1,
        tournament_size=3,
        seed=42,
    ),
    representation_model=rep.UniformRepresentationModel(),
)

t0 = time.perf_counter()
result_ga = rep.RepSetExperiment(context, workflow_ga).run()
time_ga = time.perf_counter() - t0

print(f"Selection: {result_ga.selection}")
print(f"Scores:    {result_ga.scores}")
print(f"Time:      {time_ga:.2f}s")

In [ ]:
history = result_ga.diagnostics['generation_history']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=history['generation'], y=history['best_fitness'],
    name='Best fitness', mode='lines',
))
fig.add_trace(go.Scatter(
    x=history['generation'], y=history['mean_fitness'],
    name='Mean fitness', mode='lines', line=dict(dash='dash'),
))
fig.update_layout(
    title='GA Convergence (Weighted Sum Fitness)',
    xaxis_title='Generation',
    yaxis_title='Fitness (higher = better)',
    template='plotly_white',
)
fig.show()

In [ ]:
fig = diag.ResponsibilityBars().plot(result_ga.weights, show_uniform_reference=True)
fig.update_layout(title='GA (Weighted Sum): Responsibility Weights')
fig.show()

---
## 4. Genetic Algorithm (NSGA-II Fitness)

The `NSGA2Fitness` strategy uses non-dominated sorting and crowding distance (Deb et al. 2002) to rank individuals. Instead of collapsing objectives into a single scalar, it preserves Pareto-front structure during evolution. Individuals on better (lower-rank) fronts get higher fitness, with crowding distance breaking ties within fronts.

This approach is particularly useful when objectives are not easily commensurable.

In [ ]:
workflow_nsga = rep.Workflow(
    feature_engineer=rep.StandardStatsFeatureEngineer(),
    search_algorithm=rep.GeneticAlgorithmSearch(
        objective_set=objective_set,
        selection_policy=rep.WeightedSumPolicy(),
        combination_generator=rep.ExhaustiveCombiGen(k=k),
        fitness_strategy=rep.NSGA2Fitness(),
        population_size=30,
        n_generations=50,
        mutation_rate=0.15,
        crossover_rate=0.8,
        elite_fraction=0.1,
        tournament_size=3,
        seed=42,
    ),
    representation_model=rep.UniformRepresentationModel(),
)

t0 = time.perf_counter()
result_nsga = rep.RepSetExperiment(context, workflow_nsga).run()
time_nsga = time.perf_counter() - t0

print(f"Selection: {result_nsga.selection}")
print(f"Scores:    {result_nsga.scores}")
print(f"Time:      {time_nsga:.2f}s")

In [ ]:
history_nsga = result_nsga.diagnostics['generation_history']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=history_nsga['generation'], y=history_nsga['best_fitness'],
    name='Best fitness', mode='lines',
))
fig.add_trace(go.Scatter(
    x=history_nsga['generation'], y=history_nsga['mean_fitness'],
    name='Mean fitness', mode='lines', line=dict(dash='dash'),
))
fig.update_layout(
    title='GA Convergence (NSGA-II Fitness)',
    xaxis_title='Generation',
    yaxis_title='Fitness (higher = better)',
    template='plotly_white',
)
fig.show()

---
## 5. Feature Space Comparison

Let's visualize the selections from all four methods in the same feature space.

In [ ]:
feature_ctx = rep.RepSetExperiment(context, workflow_exhaustive).feature_context
cols = list(feature_ctx.df_features.columns[:2])

fig = diag.FeatureSpaceScatter2D().plot(
    feature_ctx.df_features, x=cols[0], y=cols[1], selection=result_exhaustive.selection
)
fig.update_layout(title='Exhaustive: Feature Space')
fig.show()

In [ ]:
slicer = rep.TimeSlicer(unit="month")

selected_idx_ex = slicer.get_indices_for_slice_combi(df_raw.index, result_exhaustive.selection)
selected_idx_ga = slicer.get_indices_for_slice_combi(df_raw.index, result_ga.selection)

fig = diag.DistributionOverlayECDF().plot(df_raw['load'], df_raw.loc[selected_idx_ex, 'load'])
fig.update_layout(title='Exhaustive: ECDF Overlay (Load)')
fig.show()

fig = diag.DistributionOverlayECDF().plot(df_raw['load'], df_raw.loc[selected_idx_ga, 'load'])
fig.update_layout(title='GA (Weighted Sum): ECDF Overlay (Load)')
fig.show()

---
## Summary

All three Generate-and-Test algorithms use the same ObjectiveSet and SelectionPolicy. The only difference is how candidates are generated and explored.

In [ ]:
rows = [
    ('Exhaustive', result_exhaustive, time_exhaustive, 495),
    ('Random (N=200)', result_random, time_random, 200),
    ('GA (WeightedSum)', result_ga, time_ga, 30 * 51),
    ('GA (NSGA-II)', result_nsga, time_nsga, 30 * 51),
]

print(f"{'Method':<20} {'Selection':<45} {'Wasserstein':>12} {'Correlation':>12} {'Evals':>8} {'Time (s)':>10}")
print('-' * 110)
for name, res, t, evals in rows:
    sel_str = ', '.join(str(s) for s in res.selection)
    print(
        f"{name:<20} {sel_str:<45} "
        f"{res.scores.get('wasserstein', float('nan')):>12.4f} "
        f"{res.scores.get('correlation', float('nan')):>12.4f} "
        f"{evals:>8} "
        f"{t:>10.2f}"
    )